In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=13b38f97b9354e20770a74c018f45d6797907d5b77744f4eb0682053b9c7a069
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
#rimas - Hajer

***13.2.1. Load and clean data***


In [ ]:
#rfm is " تقنية تسويقية تعتمد على البيانات وتستخدم عادة لتجزئة العملاء ""
#"استنادًا إلى سلوكهم الشرائي السابق"

In [ ]:
from pyspark.sql import SparkSession
#لإنشاء وتكوين جلسة

spark = SparkSession \
    .builder \
    .appName("Python Spark RFM example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# load Data set

df_raw = spark.read.format('com.databricks.spark.csv').\
                       options(header='true', \
                       inferschema='true').\
            load("/content/OnlineRetail.csv",header=True);

In [ ]:
# show the ds + schema
df_raw.show(5)
df_raw.printSchema()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

root
 |-- InvoiceNo: string (nullable = true)
 |

#the data set is related to invoices for retail transactions.

the columns:

1. InvoiceNo: A unique identifier for each invoice.
2. StockCode: A code or reference number for the specific product or item.
3. Description: A textual description of the item.
4. Quantity: The quantity of the item purchased in the invoice.
5. InvoiceDate: The date and time when the invoice was generated.
6. UnitPrice: The price of a single unit of the item.
7. CustomerID: The identifier for the customer associated with the invoice.
8. Country: The country where the transaction took place.


****Data clean and data manipulation****



#check and remove the null values

my_count  حساب عدد الصفوف غير الفارغة في كل عمود في

"`count(c)` لحساب عدد الصفوف غير الفارغة في العمود المحدد `c`."
"`.alias(c)` لتسمية النتيجة المحسوبة للعمود بنفس اسم العمود."

In [ ]:
#check and remove the null values

from pyspark.sql.functions import count


def my_count(df_in):
    df_in.agg( *[ count(c).alias(c) for c in df_in.columns ] ).show()

In [ ]:
my_count(df_raw)


+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   112803|   112803|     112478|  112803|     112803|   112803|     74868| 112803|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



#D.	Data Preprocessing
إزالة الصفوف ذات القيم الناقصة (القيم الفارغة)


In [ ]:
df = df_raw.dropna(how='any')
my_count(df)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|    74868|    74868|      74868|   74868|      74868|    74868|     74868|  74868|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



انشاء عامود جديد يحتوي على قيم التاريخ و الوقت المحدد

In [ ]:
from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, lit, datediff, col


timeFmt = "MM/dd/yy HH:mm"

df = df.withColumn('NewInvoiceDate'
                 , to_utc_timestamp(unix_timestamp(col('InvoiceDate'),timeFmt).cast('timestamp')
                 , 'UTC'))

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY") #عيد سياسة تحليل الوقت والتاريخ

from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, lit, datediff, col

timeFmt = "MM/dd/yy HH:mm"

df = df.withColumn('NewInvoiceDate', to_utc_timestamp(unix_timestamp(col('InvoiceDate'), timeFmt).cast('timestamp'), 'UTC'))
# "utc"تحويل القيمة الزمنية إلى توقيت عالمي موحد

df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|     NewInvoiceDate|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|
+---------+-----

calculate total price



In [ ]:
from pyspark.sql.functions import round

df = df.withColumn('TotalPrice', round( df.Quantity * df.UnitPrice, 2 ) )# حساب إجمالي السعر لكل صف


In [ ]:
from pyspark.sql.functions import mean, min, max, sum, datediff, to_date

date_max = df.select(max('NewInvoiceDate')).toPandas()
current = to_utc_timestamp( unix_timestamp(lit(str(date_max.iloc[0][0])), \
                              'yy-MM-dd HH:mm').cast('timestamp'), 'UTC' )#تحويل البيانات الزمنية إلى توقيت عالمي موحد وحساب مدة بين التاريخ الحال


#-----------------------------------------------------_________________________________________________________________-----------------------------------------------
# Calculatre Duration
df = df.withColumn('Duration', datediff(lit(current), 'NewInvoiceDate')) #يتم حساب الفرق بين التاريخ العالمي الحالي وتواريخ العمود

In [ ]:
#يقوم بحساب ثلاثة مؤشرات رئيسية لتحليل)(recency. frequency . monetary)


recency = df.groupBy('CustomerID').agg(min('Duration').alias('Recency'))#هدف إلى قياس المدة الزمنية منذ آخر تفاعل أو عملية شراء لكل عميل.
#------------------------------------------------------------------------------------------------

frequency = df.groupBy('CustomerID', 'InvoiceNo').count()\
                        .groupBy('CustomerID')\
                        .agg(count("*").alias("Frequency")) # لتصنيف العملاء وفهم مدى تفاعلهم وتكرار عمليات الشراء

#----------------------------------------------------------------------------------------------------------------------

monetary = df.groupBy('CustomerID').agg(round(sum('TotalPrice'), 2).alias('Monetary')) # قياس القيمة المالية الإجمالية لعمليات الشراء التي قام بها كل عميل.

#---------------------------------------------------------------------------------------------------------------------------------

 # دمج البيانات و وفهم سلوكهم وقيمتهم المالية وأهمية مساهمتهم في الايراداتها
rfm = recency.join(frequency, 'CustomerID', how='inner') \
             .join(monetary, 'CustomerID', how='inner')

In [ ]:
rfm.show(5)


+----------+-------+---------+--------+
|CustomerID|Recency|Frequency|Monetary|
+----------+-------+---------+--------+
|     17389|      4|        9|  445.92|
|     15727|     54|        2|  2091.6|
|     16503|     53|        3|  831.29|
|     17420|     96|        1|  130.85|
|     15447|     53|        1|  155.17|
+----------+-------+---------+--------+
only showing top 5 rows



'''
تاثرت الايرادات بشكل جيد حيث :
  العملاء قد قاموا بتفاعل حديث، ويتكررون في الشراء بشكل متكرر،
 ويقومون بعمليات شراء ذات قيمة مالية عالية.
هذه العوامل الثلاثة يمكن أن تؤثر بشكل إيجابي على الإيرادات
-------------------------------------------------

العمود: Recency

القيم: 20, 32, 30, 8, 62
مدى التأثير: القيم الأقل تعني تفاعل أكثر حداثة وربما تكون لها تأثير أكبر على الإيرادات.

-------------------------------------------------
العمود: Frequency

القيم: 12, 4, 3, 2
مدى التأثير: القيم الأعلى تشير إلى تكرار أعلى لعمليات الشراء وبالتالي تأثير أكبر على الإيرادات.
-------------------------------------------------
العمود: Monetary

القيم: 3570.46, 384.44, 2876.05, 1173.27, 218.06
مدى التأثير: القيم الأعلى تشير إلى قيمة مالية أعلى وبالتالي تأثير أكبر على الإيرادات.

'''

***13.2.2. RFM Segmentation***


تنفيذ تحليل ل لتقييم العملاء وتصنيفهم وفقًا لسلوكهم التجاري

In [ ]:
cols = ['Recency','Frequency','Monetary']
# describe_pd(rfm,cols,1)

summary = rfm[cols].describe()



#"Recency (الحديث):  العملاء الذين قاموا بعمليات شراء حديثة أكثر قيمة وأهمية في التسويق.

#Frequency (التكرار):  العملاء الذين يقومون بعمليات شراء متكررة يمثلون عادةً نسبة كبيرة من إيرادات الشركة.

#Monetary (القيمة النقدية):  العملاء الذين يقومون بشراء منتجات أكثر قيمة أو يساهمون بمبالغ أكبر في إيرادات الشركة."

In [ ]:
summary

DataFrame[summary: string, Recency: string, Frequency: string, Monetary: string]

# حساب المؤشرات الرئيسية


In [ ]:
from pyspark.sql.functions import min, count, sum, round, avg, max

# حساب المؤشرات الرئيسية
recency_summary = rfm.agg(min('Recency').alias('min'), avg('Recency').alias('avg'), max('Recency').alias('max')).collect()
frequency_summary = rfm.agg(min('Frequency').alias('min'), avg('Frequency').alias('avg'), max('Frequency').alias('max')).collect()
monetary_summary = rfm.agg(min('Monetary').alias('min'), avg('Monetary').alias('avg'), max('Monetary').alias('max')).collect()

# طباعة المعلومات
print("Recency:")
print("Min:", recency_summary[0]['min'])
print("Avg:", recency_summary[0]['avg'])
print("Max:", recency_summary[0]['max'])
print()

print("Frequency:")
print("Min:", frequency_summary[0]['min'])
print("Avg:", frequency_summary[0]['avg'])
print("Max:", frequency_summary[0]['max'])
print()

print("Monetary:")
print("Min:", monetary_summary[0]['min'])
print("Avg:", monetary_summary[0]['avg'])
print("Max:", monetary_summary[0]['max'])

Recency:
Min: 0
Avg: 39.88828926065839
Max: 96

Frequency:
Min: 1
Avg: 2.4328116567728006
Max: 46

Monetary:
Min: -1192.2
Avg: 855.2749595250932
Max: 57821.02


يمكن ملاحظة أن المتوسطات تشير إلى أن العملاء يقومون بعمليات الشراء بشكل متوسط حوالي 2.43 مرة وتكون القيمة المالية الإجمالية حوالي 855.27.

In [ ]:
def RScore(x): #(المدة الزمنية منذ آخر تفاعل)
    if  x <= 16:
        return 1
    elif x<= 50:
        return 2
    elif x<= 143:
        return 3
    else:
        return 4


In [ ]:

def FScore(x):#(تكرار عمليات الشراء)
    if  x <= 1:
        return 4
    elif x <= 3:
        return 3
    elif x <= 5:
        return 2
    else:
        return 1


In [ ]:

def MScore(x):#(القيمة المالية الإجمالية)
    if  x <= 293:
        return 4
    elif x <= 648:
        return 3
    elif x <= 1611:
        return 2
    else:
        return 1


يمكن استخدام هذه الدرجات لتصنيف العملاء في فئات محددة بناءً على حداثة تفاعلهم، تكرار عمليات الشراء، وقيمتهم المالية.



In [ ]:
# يمكن استخدامها لتجميع العملاء في فئات مختلفة بناءً على تلك الدرجات.
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType
## تعريف الوظائف المعرفة بالنوع

R_udf = udf(lambda x: RScore(x), StringType())
F_udf = udf(lambda x: FScore(x), StringType())
M_udf = udf(lambda x: MScore(x), StringType())

إضافة ثلاثة أعمدة إضافية إلى إطار البيانات rfm تسمى "r_seg" و "f_seg" و "m_seg". تحتوي هذه الأعمدة على الدرجات المحسوبة بناءً على قيم

Recency وFrequency وMonetary على التوالي.

 الأعمدة المحسوبة لتصنيف العملاء وفهم سلوكهم وقيمتهم.

In [ ]:
rfm_seg = rfm.withColumn("r_seg", R_udf("Recency"))
rfm_seg = rfm_seg.withColumn("f_seg", F_udf("Frequency"))
rfm_seg = rfm_seg.withColumn("m_seg", M_udf("Monetary"))
#استخدام وظائف محددة للمستخدم (UDFs) منفصلة لحساب قطاعات الحداثة والتردد والنقدي (R_udf، F_udf، و M_udf).


ستخدام وظائف محددة للمستخدم (UDFs) منفصلة لحساب قطاعات الحداثة والتردد والنقدي (R_udf، F_udf، و M_udf).
إنشاء أعمدة منفصلة (r_seg، f_seg، و m_seg) لكل قطاع.

In [ ]:
rfm_seg.show(5)

+----------+-------+---------+--------+-----+-----+-----+
|CustomerID|Recency|Frequency|Monetary|r_seg|f_seg|m_seg|
+----------+-------+---------+--------+-----+-----+-----+
|     17389|      4|        9|  445.92|    1|    1|    3|
|     15727|     54|        2|  2091.6|    3|    3|    1|
|     16503|     53|        3|  831.29|    3|    3|    2|
|     17420|     96|        1|  130.85|    3|    4|    4|
|     15447|     53|        1|  155.17|    3|    4|    4|
+----------+-------+---------+--------+-----+-----+-----+
only showing top 5 rows



وجود أعمدة منفصلة (r_seg، f_seg، و m_seg) تحتوي على قطاعات محددة مسبقًا.
ربط هذه القيم معًا في عمود واحد جديد (RFMScore) باستخدام F.concat.

In [ ]:
from pyspark.sql.functions import col, concat

#Creating an RFMScore Column
rfm_seg = rfm_seg.withColumn('RFMScore', concat(col('r_seg'), col('f_seg'), col('m_seg')))

rfm_seg.sort(col('RFMScore')).show(5)#Sorting and Displaying

# rfm_seg = rfm_seg.withColumn('RFMScore',F.concat(F.col('r_seg'),F.col('f_seg'), F.col('m_seg')))
# rfm_seg.sort(F.col('RFMScore')).show(5)

+----------+-------+---------+--------+-----+-----+-----+--------+
|CustomerID|Recency|Frequency|Monetary|r_seg|f_seg|m_seg|RFMScore|
+----------+-------+---------+--------+-----+-----+-----+--------+
|     16839|      5|        9| 3238.95|    1|    1|    1|     111|
|     15570|      7|        7| 1928.54|    1|    1|    1|     111|
|     13081|      0|        8| 7377.51|    1|    1|    1|     111|
|     14180|      5|       14| 2179.67|    1|    1|    1|     111|
|     17811|      1|       10| 2936.22|    1|    1|    1|     111|
+----------+-------+---------+--------+-----+-----+-----+--------+
only showing top 5 rows



The tables show customer data categorized by how recently they bought (Recency), how often they buy (Frequency), and how much they spend (Monetary).
Scores are assigned to each category (high, medium, low).
The second table combines these scores into a single code (RFMScore) to represent a customer's overall buying behavior.

***13.2.3. Statistical Summary***


In [ ]:

rfm_seg.groupBy('RFMScore') \
  .agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean'}) \
  .sort(col('RFMScore')).show(5)

+--------+------------------+-----------------+------------------+
|RFMScore|      avg(Recency)|    avg(Monetary)|    avg(Frequency)|
+--------+------------------+-----------------+------------------+
|     111|               4.5|7060.734242424243|12.787878787878787|
|     112|               4.7|         1107.201| 7.433333333333334|
|     113|               5.0|          529.425|               8.5|
|     114|              13.0|            281.5|               7.0|
|     121|7.6923076923076925|5662.786153846154|4.6923076923076925|
+--------+------------------+-----------------+------------------+
only showing top 5 rows




 هناك اختلافات في الأداء الزمني والتكرار والقيمة المالية بين الفئات المختلفة من RFMScore. قد توفر هذه المعلومات
  فهمًا أوليًا لنمط سلوك
 العملاء وتساعد في اتخاذ قرارات استراتيجية

  مثل استهداف العملاء ذوي RFMScore محددة لزيادة الربحية وتعزيز التسويق.



''' Regarding Recency Performance: Customers with a low RFMScore have lower Recency periods,
which indicates that they have made recent purchases. Conversely, customers with a higher RFMScore
have greater Recency Periods, meaning they made purchases over a longer period of time.


Regarding frequency: There is a difference in the average frequency between the different
categories of RFMScore. Customers with higher RFMScore values appear to make less frequent
 purchases compared to customers with lower RFMScore values. This means that customers with a
  higher RFMScore may be exclusivity customers or customers who make infrequent purchases.


Regarding Monetary value: A significant difference can be observed in the financial
value of customers in different categories of RFMScore. For example, customers with an
RFMScore of "111" have a high average equity value of 7060.734242424243, while customers
with an RFMScore of "112" have a lower average equity value of 1107.201. This means that customers
with higher RFMScore values may be customers with lower financial value. '''

# 1. Recency:
   - Identify customers with a low recency score (1) as they are more likely to be high-value customers who have made recent purchases.
   - Customers with a high recency score (4) may be at-risk customers who have not made purchases in a long time.

#2. Frequency:
   - Customers with a high frequency score (1) are loyal and frequent buyers, representing high-value customers.
   - Low frequency score (4) indicates customers who make infrequent purchases and may be at-risk.

#3. Monetary:
   - High monetary score (1) signifies customers who spend the most and are valuable to the business.
   - Low monetary score (4) suggests customers who spend less and may need targeted marketing efforts to increase their spending.

اغلاق الجلسه

In [ ]:
spark.stop()